In [1]:
# Biblotecas de manipulação de dados
import numpy as np
import pandas as pd
# Bibliotecas de visualização
import seaborn as sns
import matplotlib.pyplot as plt
# Testes estatísticos
from scipy.stats import ttest_ind

from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import datasets
from sklearn.metrics import classification_report 

# Bibliotecas Locais
from carga import *
from tratamentos import *
from categorizacao import *

pfEnade = doCarga('microdados_enade_2019.txt')
# TRATAMENTOS 
pfEnade = doIdade(pfEnade)
pfEnade = doNota(pfEnade)
pfEnade = doVazios(pfEnade)
pfEnade = doPublicoPrivada(pfEnade)
pfEnade = doHumanasExatas(pfEnade)
pfEnade = doRenda(pfEnade)
pfEnade = doMigracao(pfEnade)
pfEnade = doRenda(pfEnade)
pfEnade = doEscolaridade(pfEnade)
pfEnade = doCategoricos(pfEnade)

In [2]:
# CONTAR CLASSES
count_class_0, count_class_1 = pfEnade['Publica'].value_counts()# pfEnade.target.value_counts()
# Divide by class
df_class_0 = pfEnade[pfEnade['Publica'] == 1]   # 1 = Privada
df_class_1 = pfEnade[pfEnade['Publica'] == 0]   # 0 = Publica
df_class_0_under = df_class_0.sample(count_class_1)
dfAjustado = pd.concat([df_class_0_under, df_class_1], axis=0)

dfAjustado['Publica'].value_counts() 

0    41664
1    41664
Name: Publica, dtype: int64

In [3]:
# CARGA - X e y
y = pd.DataFrame(dfAjustado, columns = ['Publica'])
enade_features = [ 'BomAluno','CO_REGIAO_CURSO', 'Jovem', 'Migrado', 'Solo', 'Exatas', 'QE_I23CAT', 'QE_I08CAT', 'Sustento', 'Sexo'] 
# , 'Familia', , 'NU_IDADE' BaixaRenda
# , 'Tempo', 'RecemFormado', 'QE_I08CAT', 'Age'
# Ou solo ou família

X = dfAjustado[enade_features]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [4]:
X_train.shape

(55829, 10)

In [5]:
X_test.shape 

(27499, 10)

In [ ]:
# Plotar em pares 
# tentar visualizar se é possível um divisão linear (no caso não é)
#sns.pairplot(pfEnade[['QE_I08CAT', 'QE_I23CAT', 'NT_GER']], hue='QE_I08CAT')
plot_features = [ 'BomAluno','CO_REGIAO_CURSO', 'Jovem', 'Migrado', 'Solo', 'Exatas', 'QE_I23CAT', 'QE_I08CAT', 'Sustento', 'Sexo', 'Publica'] 
sns.pairplot(pfEnade[plot_features], hue='Publica')

In [6]:
# SVM - Support Vector Machines
# Separar classes por linha reta
C=1.0   # marge 
clf = svm.SVC(kernel='linear', C=C)

In [11]:
clf.fit(X_train, y_train.values.ravel())

SVC(kernel='linear')

In [12]:
y_pred = clf.predict(X_test)

In [14]:
clf.score(X_test, y_test)

0.6166042401541875

In [15]:
# Relatório de Classificação F1
# print (classification_report(y_test, y_pred))
target_names=(doPublicaDIC())
print (classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

           0       0.61      0.63      0.62     13725
           1       0.62      0.60      0.61     13774

    accuracy                           0.62     27499
   macro avg       0.62      0.62      0.62     27499
weighted avg       0.62      0.62      0.62     27499

